In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### **Section 1: Setup**

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [3]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("ADK component imported!!!")

ADK component imported!!!


In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429,500,503,504]
)

## **creation multi agent sys**


## **Sequential Workflows - The Assembly Line**

### *‼️ However, relying on an LLM's instructions to control the order can sometimes be unpredictable. Next, we'll explore a different pattern that gives you guaranteed, step-by-step execution.*

#### Blog Post Creation with Sequential Agents
 Let's build a system with three specialized agents:

- Outline Agent - Creates a blog outline for a given topic
- Writer Agent - Writes a blog post
- Editor Agent - Edits a blog post draft for clarity and structure

In [5]:
# outline agent
outline_agent=Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_option=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline"
)

# writer agent
writer_agent=Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_option=retry_config
    ),
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft"
)

# editor agent
editor_agent=Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_option=retry_config
    ),
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog"
)

print("worker agents are created!!!")

worker agents are created!!!


In [6]:
root_agent=SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

In [7]:
runner=InMemoryRunner(agent=root_agent)
response= await runner.run_debug(
    "Write a blog post about the benefits of multi-agent systems for software developers"
)


 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers
OutlineAgent > ### Blog Outline:

**Headline:** Unleash Superpowers: How Multi-Agent Systems Are Revolutionizing Software Development

**Introduction Hook:** Imagine a team of intelligent, specialized digital assistants working tirelessly behind the scenes, optimizing your code, identifying bugs before they happen, and even suggesting innovative solutions. This isn't science fiction; it's the power of Multi-Agent Systems (MAS), and they're poised to transform how we build software.

**Main Sections:**

**1. Smarter Development, Faster Delivery:**

*   **Automated Code Review & Optimization:** MAS agents can continuously analyze code, pinpointing inefficiencies, potential bugs, and stylistic inconsistencies, freeing up developers for more complex tasks.
*   **Proactive Bug Detection & Prevention:** By learning from historical data and real-time monitori